In [78]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Imports
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adadelta, Adagrad, Adam
from keras.utils import np_utils, generic_utils
from keras import backend as K
from keras.layers.core import Lambda
from six.moves import range
import numpy as np
import scipy as sp
#from keras import backend as K
import random
import scipy.io
import matplotlib.pyplot as plt
from keras.regularizers import l2
import math
import numpy as np
import sys
import random
import time
import os
import threading
from tensorflow.python.framework import ops
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers import Dense, Conv1D, SpatialDropout1D, MaxPooling1D, MaxPooling2D,Input, Flatten
from keras import models
from keras import optimizers
import numpy as np
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
import keras
import numpy as np
import time
import sys
import keras.backend as K
from keras import initializers
from keras.engine import InputSpec
from keras.layers import Dense, Lambda, Wrapper
import tensorflow as tf 
from keras.models import Model

nuc_arr = ['A','C','G','T']
#Function for calculating modified probability of splicing at SD1
def prob_SD1 (sd1_freq, sd2_freq):
    if (sd1_freq==0 and sd2_freq==0):
        return 0.0
    else:
        return sd1_freq/(sd1_freq+sd2_freq)
#Function converting nucleotide sequence to numerical array with 4 channels
def seq_to_arr (seq):
    seq_len = len(seq)
    arr_rep = np.zeros((seq_len, len(nuc_arr)))
    for i in range(seq_len):
        arr_rep[i][nuc_arr.index(seq[i])] = 1
    return arr_rep
  
def permanent_dropout_layer(rate):
    seed = np.random.randint(0,100)
    return Lambda(lambda x: K.dropout(x,level=rate,seed=seed))

#Creating a modified dataset with only the necessary information
#Storing model inputs and outputs
reads_path = "/content/gdrive/My Drive/Active Learning Research 2019/GSM1911086_A5SS_spliced_reads.txt"
seq_path = "/content/gdrive/My Drive/Active Learning Research 2019/GSM1911085_A5SS_seq.txt"
s1_indx = 1
s2_indx= 45
seq_len = 101
read_lines = []
seq_lines = []
data_table = []
with open(reads_path) as f:
    f.readline()
    for line in f:
        mod_line = line.split('\t')
        read_lines.append([mod_line[0], mod_line[s1_indx], mod_line[s2_indx]])
with open(seq_path) as f:
    f.readline()
    for line in f:
        mod_line = line.split('\t')
        seq_lines.append([mod_line[0], mod_line[1][:-1]])

n = len(read_lines)
prob_s1 = np.zeros(n)
inputs = np.zeros((n,seq_len, 4))

for i in range(n):
    prob_s1[i] = prob_SD1(float(read_lines[i][1]), float(read_lines[i][2]))
    inputs[i] = seq_to_arr(seq_lines[i][1])



In [0]:

#From Concrete Dropout Paper, on GitHub
class ConcreteDropout(Wrapper):
    """This wrapper allows to learn the dropout probability for any given input Dense layer.
    ```python
        # as the first layer in a model
        model = Sequential()
        model.add(ConcreteDropout(Dense(8), input_shape=(16)))
        # now model.output_shape == (None, 8)
        # subsequent layers: no need for input_shape
        model.add(ConcreteDropout(Dense(32)))
        # now model.output_shape == (None, 32)
    
    `ConcreteDropout` can be used with arbitrary layers which have 2D
    kernels, not just `Dense`. However, Conv2D layers require different
    weighing of the regulariser (use SpatialConcreteDropout instead).
    # Arguments
        layer: a layer instance.
        weight_regularizer:
            A positive number which satisfies
                $weight_regularizer = l**2 / (\tau * N)$
            with prior lengthscale l, model precision $\tau$ (inverse observation noise),
            and N the number of instances in the dataset.
            Note that kernel_regularizer is not needed.
        dropout_regularizer:
            A positive number which satisfies
                $dropout_regularizer = 2 / (\tau * N)$
            with model precision $\tau$ (inverse observation noise) and N the number of
            instances in the dataset.
            Note the relation between dropout_regularizer and weight_regularizer:
                $weight_regularizer / dropout_regularizer = l**2 / 2$
            with prior lengthscale l. Note also that the factor of two should be
            ignored for cross-entropy loss, and used only for the eculedian loss.
    """

    def __init__(self, layer, weight_regularizer=1e-6, dropout_regularizer=1e-5,
                 init_min=0.1, init_max=0.1, is_mc_dropout=True, **kwargs):
        assert 'kernel_regularizer' not in kwargs
        super(ConcreteDropout, self).__init__(layer, **kwargs)
        self.weight_regularizer = weight_regularizer
        self.dropout_regularizer = dropout_regularizer
        self.is_mc_dropout = is_mc_dropout
        self.supports_masking = True
        self.p_logit = None
        self.p = None
        self.init_min = np.log(init_min) - np.log(1. - init_min)
        self.init_max = np.log(init_max) - np.log(1. - init_max)

    def build(self, input_shape=None):
        self.input_spec = InputSpec(shape=input_shape)
        if not self.layer.built:
            self.layer.build(input_shape)
            self.layer.built = True
        super(ConcreteDropout, self).build()  # this is very weird.. we must call super before we add new losses

        # initialise p
        self.p_logit = self.layer.add_weight(name='p_logit',
                                            shape=(1,),
                                            initializer=initializers.RandomUniform(self.init_min, self.init_max),
                                            trainable=True)
        self.p = K.sigmoid(self.p_logit[0])

        # initialise regulariser / prior KL term
        assert len(input_shape) == 2, 'this wrapper only supports Dense layers'
        input_dim = np.prod(input_shape[-1])  # we drop only last dim
        weight = self.layer.kernel
        kernel_regularizer = self.weight_regularizer * K.sum(K.square(weight)) / (1. - self.p)
        dropout_regularizer = self.p * K.log(self.p)
        dropout_regularizer += (1. - self.p) * K.log(1. - self.p)
        dropout_regularizer *= self.dropout_regularizer * input_dim
        regularizer = K.sum(kernel_regularizer + dropout_regularizer)
        self.layer.add_loss(regularizer)

    def compute_output_shape(self, input_shape):
        return self.layer.compute_output_shape(input_shape)

    def concrete_dropout(self, x):
        '''
        Concrete dropout - used at training time (gradients can be propagated)
        :param x: input
        :return:  approx. dropped out input
        '''
        eps = K.cast_to_floatx(K.epsilon())
        temp = 0.1

        unif_noise = K.random_uniform(shape=K.shape(x))
        drop_prob = (
            K.log(self.p + eps)
            - K.log(1. - self.p + eps)
            + K.log(unif_noise + eps)
            - K.log(1. - unif_noise + eps)
        )
        drop_prob = K.sigmoid(drop_prob / temp)
        random_tensor = 1. - drop_prob

        retain_prob = 1. - self.p
        x *= random_tensor
        x /= retain_prob
        return x

    def call(self, inputs, training=None):
        if self.is_mc_dropout:
            return self.layer.call(self.concrete_dropout(inputs))
        else:
            def relaxed_dropped_inputs():
                return self.layer.call(self.concrete_dropout(inputs))
            return K.in_train_phase(relaxed_dropped_inputs,
                                    self.layer.call(inputs),
                                    training=training)


In [0]:
#Experiment Settings
num_experiments = 1
num_acquisitions = 100
bald_sample_size = 5000
dropout_iterations = 100
size_train = 400
dropout = 0.1
batchsize = 128

In [0]:
#Loading Data
np.random.seed(1)
X = np.array(inputs)
y = np.array(prob_s1)

In [83]:
#Model with Spatial and Concrete Dropout

tau = 0.159708
N = X_train.shape[0]
lengthscale = 1e2
Weight_Decay = (1 - dropout) / (2. * N * lengthscale**2 * tau)
inputlayer = Input(shape=(seq_len, len(nuc_arr)))
x = Conv1D(seq_len,(4), strides=1, input_shape=(seq_len,len(nuc_arr)), activation='relu')(inputlayer)
x = SpatialDropout1D(dropout)(x)
x = MaxPooling1D(pool_size=3)(x)
x = permanent_dropout_layer(dropout)(x)
x = Conv1D(seq_len//2, (4), strides=1, activation='relu')(x)
x = SpatialDropout1D(dropout)(x)
x = MaxPooling1D(pool_size=3)(x)
x = Flatten()(x)
x = permanent_dropout_layer(dropout)(x)
x = ConcreteDropout(Dense(50, W_regularizer=l2(Weight_Decay), init='normal',  activation='relu'), input_shape=(450,))(x)
outputlayer = ConcreteDropout(Dense(1, W_regularizer=l2(Weight_Decay), init='normal'), input_shape=(50,))(x)
model_rand = Model(inputlayer, outputlayer)
model_bald = Model(inputlayer, outputlayer)

def predict_with_uncertainty(f, x, n_iter=100):
    result = np.zeros((n_iter,x.shape[0]))
    for i in range(n_iter):
        predictions = np.array(f((x, 1))[0])[:,0]
        result[i,:] = predictions
    prediction = result.mean(axis=0)
    uncertainty = result.std(axis=0)
    return prediction, uncertainty    

model_rand.compile(loss='mean_squared_error', optimizer='rmsprop')
model_bald.compile(loss='mean_squared_error', optimizer='rmsprop')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, activation="relu", kernel_initializer="normal", kernel_regularizer=<keras.reg...)`
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, kernel_initializer="normal", kernel_regularizer=<keras.reg...)`


In [84]:
#Active Learning Experiments
all_mse_rand = []
all_mse_bald = []
for exp in range(num_experiments):
    #Training initial model
    all_train_indices = np.load('/content/gdrive/My Drive/Active Learning Research 2019/Train-Test-Splits/trainindices' + str(exp+1) + '.npy')
    all_train_indices_bald = np.load('/content/gdrive/My Drive/Active Learning Research 2019/Train-Test-Splits/trainindices' + str(exp+1) + '.npy')
    train_data_indices_rand = all_train_indices[0:size_train].tolist()
    test_indices = np.load('/content/gdrive/My Drive/Active Learning Research 2019/Train-Test-Splits/testindices' + str(exp+1) + '.npy')
    X_train_rand = X[train_data_indices_rand]
    y_train_rand = y[train_data_indices_rand]
    X_test = X[test_indices]
    y_test = y[test_indices]
    model_rand.fit(X_train_rand, y_train_rand, epochs=50, batch_size=batchsize, verbose=0)
    f_rand = K.function([model_rand.layers[0].input, K.learning_phase()],
        [model_rand.layers[-1].output])
    predictions_with_uncertainty = predict_with_uncertainty(f_rand, X_test, n_iter=100)
    y_predicted = predictions_with_uncertainty[0]
    y_uncertainty = predictions_with_uncertainty[1]
    mse = np.mean((y_test - y_predicted)**2)
    exp_mse_rand = []
    exp_mse_bald = []
    exp_mse_rand.append(mse)
    exp_mse_bald.append(mse)
    #Initializing data, MSE, and model for BALD
    train_data_indices_bald = all_train_indices[0:size_train].tolist()
    X_train_bald =  X[train_data_indices_bald]
    y_train_bald = y[train_data_indices_bald]
    model_bald.fit(X_train_bald, y_train_bald, epochs=50, batch_size=batchsize, verbose=0)
    f_bald = K.function([model_bald.layers[0].input, K.learning_phase()], 
        [model_bald.layers[-1].output])
    #Initializing pool indices
    pool_indices_rand = [ind for ind in all_train_indices if ind not in train_data_indices_rand]
    #Data Acquisitions
    for acq in range(num_acquisitions):
        #Add a random point to train_data_indices_rand
        pool_ind_ind_rand = np.random.choice(len(pool_indices_rand))
        data_ind_rand = pool_indices_rand[pool_ind_ind_rand]
        train_data_indices_rand.append(data_ind_rand)
        X_train_rand = X[train_data_indices_rand]
        y_train_rand = y[train_data_indices_rand] 
        del pool_indices_rand[pool_ind_ind_rand]
        #Fitting RandAcq Model and calculating MSE
        model_rand.fit(X_train_rand, y_train_rand, epochs=50, batch_size=batchsize, verbose=0)
        f_rand = K.function([model_rand.layers[0].input, K.learning_phase()],
           [model_rand.layers[-1].output])
        predictions_with_uncertainty = predict_with_uncertainty(f_rand, X_test, n_iter=100)
        y_predicted = predictions_with_uncertainty[0]
        mse = np.mean((y_test - y_predicted)**2)
        exp_mse_rand.append(mse)     
        #Adding max variance point of a pool sample to train_data_indices_bald
        pool_indices_bald = np.array([ind for ind in all_train_indices_bald if ind not in train_data_indices_bald])
        pool_indices_indices_sample = np.random.choice(len(pool_indices_bald), bald_sample_size, replace=False).astype(int)
        random_sample_indices = pool_indices_bald[pool_indices_indices_sample]
        X_random_sample = X[random_sample_indices]
        predictions_with_uncertainty = predict_with_uncertainty(f_bald, X_random_sample, n_iter=100)
        y_uncertainty = predictions_with_uncertainty[1]
        new_data_ind = random_sample_indices[np.argmax(y_uncertainty)]
        train_data_indices_bald.append(new_data_ind)
        X_train_bald = X[train_data_indices_bald]
        y_train_bald = y[train_data_indices_bald]
        #Fitting new BALD model and calculating MSE
        model_bald.fit(X_train_bald, y_train_bald, epochs=50, batch_size=batchsize, verbose=0)
        f_bald = K.function([model_bald.layers[0].input, K.learning_phase()], 
            [model_bald.layers[-1].output])
        predictions_with_uncertainty = predict_with_uncertainty(f_bald, X_test, n_iter=100)
        y_predicted = predictions_with_uncertainty[0]
        mse = np.mean((y_test - y_predicted)**2)
        exp_mse_bald.append(mse)
        if acq % 5 == 4:
            print('iteration ' + str(acq+1))
            print('rand mse: ' + str(exp_mse_rand))
            print('bald mse: ' + str(exp_mse_bald))
    #Storing Experiment Results
    all_mse_rand.append(exp_mse_rand)
    all_mse_bald.append(exp_mse_bald)
print('all_mse_rand: ' + str(all_mse_rand))
print('all_mse_bald: ' + str(all_mse_bald))
filepath_bald = "/content/gdrive/My Drive/Active Learning Research 2019/BaldMSE.npy"
filepath_rand = "/content/gdrive/My Drive/Active Learning Research 2019/RandMSE.npy"
np.save(filepath_bald, all_mse_bald)
np.save(filepath_rand, all_mse_rand)

iteration 5
rand mse: [0.09680930363156326, 0.09019688634526189, 0.09616766017621396, 0.09352515661295151, 0.0872482598127092, 0.0913719057882612]
bald mse: [0.09680930363156326, 0.09653542432303903, 0.09453968947668599, 0.09267396005658671, 0.09924173532084934, 0.09057427814716583]
iteration 10
rand mse: [0.09680930363156326, 0.09019688634526189, 0.09616766017621396, 0.09352515661295151, 0.0872482598127092, 0.0913719057882612, 0.0914387027592231, 0.095992536292725, 0.09706464150383248, 0.08788998350312548, 0.09100847595367496]
bald mse: [0.09680930363156326, 0.09653542432303903, 0.09453968947668599, 0.09267396005658671, 0.09924173532084934, 0.09057427814716583, 0.09098186564739935, 0.09114612419609712, 0.0982413650788339, 0.0931574740706653, 0.09207465094273729]
iteration 15
rand mse: [0.09680930363156326, 0.09019688634526189, 0.09616766017621396, 0.09352515661295151, 0.0872482598127092, 0.0913719057882612, 0.0914387027592231, 0.095992536292725, 0.09706464150383248, 0.0878899835031254